In [1]:
import os
dir_notebook = os.getenv( "HOME" )
file_include = dir_notebook + "/_include.ipynb"
%run $file_include

**Define/make directories**

In [2]:
result_dir = "result/"
#os.mkdir(result_dir)

usearch_dir = "/home/j1lim/Usearch/usearch_10"

**Input genome files**

In [3]:
dir_genomes = "/home/j1lim/BPGA_genome/temp/"
lst_genomes = os.listdir(dir_genomes)
lst_genomes.sort()

print len(lst_genomes)
print lst_genomes[:5]

5
['01_K10.fas', '02_4B.fas', '03_2015WD_1.fas', '04_2015WD_2.fas', '05_A3.fas']


**Make total input seq/info file**

In [4]:
input_all_info = "input_all_original.seq"
input_all_seq = input_all_info.replace(".seq", ".fas")
input_data_info = "dataset.tsv"

f_info = open(input_all_info, "w")
f_seq = open(input_all_seq, "w")
f_data = open(input_data_info, "w")
f_data.write("Genome_No.\tOrganism_name\tFile_name\r\n")

org_num = 1
for x in lst_genomes:
    dir_file = dir_genomes + x
    to_write = "%s\tOrganism%s\t%s\r\n" % (org_num, org_num, x)
    f_data.write(to_write)
    
    gene_num = 1
    for y in SeqIO.parse(dir_file, "fasta"):
        to_write_info = ">%s|Gene%s|%s|>%s\r\n" % (org_num, gene_num, x, y.description)
        to_write_seq = ">%s|Org%s_Gene%s|%s#\r\n%s\r\n" % (org_num, org_num, gene_num, x, str(y.seq))
        
        f_info.write(to_write_info)
        f_seq.write(to_write_seq)
        
        gene_num += 1
        #break
    org_num += 1
    #break

f_info.close()
f_seq.close()
f_data.close()

**All input clustering**

In [5]:
identity_ = 0.9
coverage_ = 0.3
centroids_out = "temp_centroids.fas"
cluster_detail = centroids_out.replace(".fas", ".uc")

cluster_command = "%s -cluster_fast %s -id %s -centroids %s -uc %s -query_cov %s -target_cov %s -maxaccepts 0 -maxrejects 0 -sort length" % (
                    usearch_dir, input_all_seq, identity_, centroids_out, cluster_detail, coverage_, coverage_)
print cluster_command
!$cluster_command

/home/j1lim/Usearch/usearch_10 -cluster_fast input_all_original.fas -id 0.9 -centroids temp_centroids.fas -uc temp_centroids.uc -query_cov 0.3 -target_cov 0.3 -maxaccepts 0 -maxrejects 0 -sort length
usearch v10.0.240_i86linux32, 4.0Gb RAM (11.1Gb total), 4 cores
(C) Copyright 2013-17 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: limjwlove@nate.com

00:00 48Mb    100.0% Reading input_all_original.fas
00:00 44Mb    100.0% DF                            
00:00 44Mb   23914 seqs, 7061 uniques, 2177 singletons (30.8%)
00:00 44Mb   Min size 1, median 4, max 27, avg 3.39
00:00 48Mb    100.0% DB
00:00 48Mb   Sort length... done.
00:06 160Mb   100.0% 5165 clusters, max size 28, avg 4.6
00:06 160Mb   100.0% Writing centroids to temp_centroids.fas
                                                            
      Seqs  7061
  Clusters  5165
  Max size  28
  Avg size  4.6
  Min size  1
Singletons  374, 5.3% of seqs, 7.2% of clusters
   Max mem  160Mb
      Time  6.00s
T

**Cluster results summary**

In [6]:
sort_command = "%s -sortbylength %s -fastaout %s" % (usearch_dir, centroids_out, centroids_out.replace("temp_", ""))
print sort_command
!$sort_command

/home/j1lim/Usearch/usearch_10 -sortbylength temp_centroids.fas -fastaout centroids.fas
usearch v10.0.240_i86linux32, 4.0Gb RAM (11.1Gb total), 4 cores
(C) Copyright 2013-17 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: limjwlove@nate.com

00:00 42Mb    100.0% Reading temp_centroids.fas
00:00 8.2Mb  Sorting by length                 
00:00 8.2Mb  Length min 20, median 263, max 6385
00:00 8.2Mb   100.0% Writing output
00:00 8.2Mb  Write done, closing file and exiting


In [7]:
pd_cluster = pd.read_csv(cluster_detail, header=-1, sep="\t")
pd_cluster = pd_cluster.sort_values(by=[1, 0], ascending=[True, True])
pd_cluster.to_csv(cluster_detail.replace("temp_", ""), sep="\t", index=False, header=False)

**Make matrix file**

In [8]:
f = open(input_data_info, "r")
r = f.readlines()
f.close()

dic_org_num = {}
for x in r:
    if x.startswith("Genome_No"):
        continue
    
    x = x.replace("\r\n", "")
    spl_ = x.split("\t")
    dic_org_num[spl_[-1]] = int(spl_[0]) -1
    #break
    
print "dic_org_num: ", len(dic_org_num)
for k, v in dic_org_num.items():
    print "Key: ", k
    print "Value: ", v
    break

dic_org_num:  5
Key:  01_K10.fas
Value:  0


In [9]:
f = open(cluster_detail.replace("temp_", ""), "r")
r = f.readlines()
f.close()

matrix_file = "matrix.tsv"
f_matrix = open(matrix_file, "w")

temp_matrix = ["0" for k in dic_org_num.keys()]
for x in r:
    x = x.replace("\n", "")
    spl_ = x.split("\t")
    
    if spl_[0] == "S":
        #print temp_matrix
        f_matrix.write("\t".join(temp_matrix) + "\r\n")
        temp_matrix = ["0" for k in dic_org_num.keys()]
        continue
    
    temp_strain = spl_[8].split("|")[-1][:-1]
    temp_matrix[dic_org_num[temp_strain]] = "1"
    
f_matrix.close()

**Make pan-genome REPSEQ fasta file**

In [10]:
f = open(matrix_file, "r")
r = f.readlines()
f.close()

dic_gene_distribution = {}
temp_num = 0
for x in r:
    x = x.replace("\r\n", "")
    spl_ = x.split("\t")
    
    if spl_.count("1") == len(spl_):
        dic_gene_distribution[temp_num] = "CORE"
    elif spl_.count("1") == 1:
        dic_gene_distribution[temp_num] = "UNIQUE"
    else:
        dic_gene_distribution[temp_num] = "ACCESSORY"
    
    temp_num += 1
    #break
    
print "dic_gene_dicstibution:", len(dic_gene_distribution)
for k, v in dic_gene_distribution.items():
    print "Key: ", k
    print "Value: ", v
    break

dic_gene_dicstibution: 5165
Key:  0
Value:  ACCESSORY


In [11]:
rep_core = "REPSEQ_CORE.fas"
rep_acce = "REPSEQ_ACCESSORY.fas"
rep_uniq = "REPSEQ_UNIQUE.fas"

f_rep_core = open(rep_core, "w")
f_rep_acce = open(rep_acce, "w")
f_rep_uniq = open(rep_uniq, "w")

temp_num = 0
for x in SeqIO.parse(centroids_out.replace("temp_", ""), "fasta"):
    distribution = dic_gene_distribution[temp_num]
    spl_ = x.description.split("|")
    to_write = ">%s_REP|%s|%s\r\n%s\r\n" % (distribution, temp_num, spl_[1], str(x.seq))
    
    if distribution == "CORE":
        f_rep_core.write(to_write)
    elif distribution == "ACCESSORY":
        f_rep_acce.write(to_write)
    else:
        f_rep_uniq.write(to_write)
    
    temp_num += 1
    #break
    
f_rep_core.close()
f_rep_acce.close()
f_rep_uniq.close()

**Make pan-genome all fasta file**

In [12]:
dic_gene_seq_info = {}
for x in SeqIO.parse(input_all_seq, "fasta"):
    dic_gene_seq_info[x.description] = str(x.seq)
    #break
    
print "dic_gene_seq_info:", len(dic_gene_seq_info)
for k, v in dic_gene_seq_info.items():
    print "Key: ", k
    print "Value: ", v
    break

dic_gene_seq_info: 23914
Key:  2|Org2_Gene1001|02_4B.fas#
Value:  MTPRPESLASSPQPVERRGSRRGFRPDIEGLRAVAVIAVVLYHAGIPGIGGGYIGVDVFFVISGFLITGLLFREASTTNTVALGRFYGARARRLLPAAAIVGTVTAIGAAAVLPPLQARSVFLDGVASALYVGNYRFALRGTDYLTADAPSPFQHYWSLGVEEQFYLVWPLLIIAVAASVRRVRPLRATPRPVPYAVVLALIGAASLVAGVLWTRTSPSWAFFSLPTRAWELAAGGLVALSIQQWRRLSLRTASIARWGGLALILLTCTQLNAHTPYPGTEALLPVLGTALIIGGGAVTGGLGPGRLLCRPAMRAIGRISYSWYLWHWPVLLLMPALLGEPAGLPARLSATAVSAGLAVLTLHLVENPGRFAAALRRSAKVSLVVAGGASAATACACLVLLNVIPAPVGHGVAAPRANIIATPPSNVPAISAQEAAVRQAFAQARVILAAAAGVRAVPSNLDPPLAQAPANKAAVFVNGCVRSWRDVGQSECATGDTASPTTVALIGDSHAAMWNPAFEQLAGQRHWRLETLAKVTCPLAELHIVSPYLGREYTECEQWRAQIMARMNAEHPRLVVLDMSRRYGADFGFTSYDPAWIDTLGRTVAQLRGTGATVLVLGPSPDPHSSAPVCLSAHLDDVTACAPTRSAAIDGDGVAAEQAATLGNGGHYADLSDLFCTPDRCPLIVGNTLVFRDDNHVTTEYARLLAPVLGALADRALAEG


In [13]:
f = open(input_data_info, "r")
r = f.readlines()
f.close()

temp_lst = []
for x in r:
    if x.startswith("Genome_No."):
        continue
    x = x.replace("\r\n", "")
    spl_ = x.split("\t")
    temp_lst.append(spl_[-1])
print len(temp_lst)
print temp_lst

5
['01_K10.fas', '02_4B.fas', '03_2015WD_1.fas', '04_2015WD_2.fas', '05_A3.fas']


In [14]:
all_core = "core_seq.fas"
all_acce = "accessory_seq.fas"
all_uniq = "unique_seq.fas"

f_all_core = open(all_core, "w")
f_all_acce = open(all_acce, "w")
f_all_uniq = open(all_uniq, "w")


f = open(cluster_detail.replace("temp_", ""), "r")
r = f.readlines()
f.close()

for x in temp_lst:

    for y in r:
        y = y.replace("\n", "")
        spl_ = y.split("\t")
        if spl_[0] == "C":
            continue
        if x not in spl_[-2]:
            continue
            
        distribution = dic_gene_distribution[int(spl_[1])]
        to_write = ">%s/%s/%s[%s]\r\n%s\r\n" % (distribution.lower(), spl_[1], 
                   spl_[-2].split("|")[1], spl_[2], dic_gene_seq_info[spl_[-2]])
        
        if distribution == "CORE":
            f_all_core.write(to_write)
        elif distribution == "ACCESSORY":
            f_all_acce.write(to_write)
        else:
            f_all_uniq.write(to_write)

        #break
    #break

f_all_core.close()
f_all_acce.close()
f_all_uniq.close()